# Applied Data Science Capstone Week 3 Assignment

## Segmenting and Clustering Neighborhoods in Toronto

##### 1. Firstly, import some required library

In [169]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't complete
import folium

##### 2. Parsing table from HTML and create data frame

In [170]:
html_data = pd.read_html(canadaPostalCodeHtmlURL)
canadaPostalCodeDF = html_data[0]
#canadaPostalCodeDF

##### 3. Remove row having Borough value as 'Not assigned'

In [171]:
boroughNotAssignedIndexes = canadaPostalCodeDF[canadaPostalCodeDF['Borough'] == "Not assigned"].index
canadaPostalCodeDF.drop(boroughNotAssignedIndexes , inplace=True)
#canadaPostalCodeDF

##### 4. As more than one neighborhood can exist in one postal code area, merge neighborhood value based on the postal code area

In [172]:
canadaPostalCodeDF = canadaPostalCodeDF.groupby(['Postcode'], as_index = False).agg({'Borough':'first', 'Neighbourhood': ', '.join})
#canadaPostalCodeDF

##### 5. If Neighbourhood is 'Not assigned' while Borough has value, copy Borough value to Neighborhood

In [173]:
neighbourhoodNotAssignedIndexes = canadaPostalCodeDF[canadaPostalCodeDF['Neighbourhood'] == "Not assigned"].index
if (len(neighbourhoodNotAssignedIndexes) > 0):
    for idx in neighbourhoodNotAssignedIndexes:
        canadaPostalCodeDF.iloc[idx,2] = canadaPostalCodeDF.iloc[idx,1]

##### 6. Now use .shape method for printing the number of rows of the dataframe

In [174]:
canadaPostalCodeDF.shape
#canadaPostalCodeDF

(103, 3)

##### 7. Merge Longitude and Latitude for postal code

In [175]:
postalCodeGeoCoorURL = "http://cocl.us/Geospatial_data"
postalCodeGeoCoorDF = pd.read_csv(postalCodeGeoCoorURL)
postalCodeGeoCoorDF.head()

finalCanadaPostalCodeDF = pd.concat([canadaPostalCodeDF, postalCodeGeoCoorDF], axis=1, join='inner')
finalCanadaPostalCodeDF = finalCanadaPostalCodeDF.drop(columns=['Postal Code'])
finalCanadaPostalCodeDF.columns = ['PostalCode','Borough','Neighbourhood','Latitude','Longitude']
finalCanadaPostalCodeDF

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


##### 8. Explore and cluster the neighborhoods in Toronto

In [181]:
map_Toronto = folium.Map(location=[43.806686, -79.194353], zoom_start=11)

# add markers to map
for lat, lng, label in zip(finalCanadaPostalCodeDF['Latitude'], finalCanadaPostalCodeDF['Longitude'], finalCanadaPostalCodeDF['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto